In [ ]:
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2

from sklearn import preprocessing


import xml.etree.ElementTree as ET

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import SequentialSampler

import glob
import cv2
from PIL import Image

In [ ]:
!pip3 install utils

In [ ]:
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
from utils import *
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
firstDroneVideoPath = "/content/drive/MyDrive/DroneDetection/Drone Tracking 1.mp4"
secondDroneVideoPath = "/content/drive/MyDrive/DroneDetection/Drone Tracking 2.mp4"
primaryDatasetPath = "/content/drive/MyDrive/DroneDetection/faster-rcnn-drone-detection-pytorch.ipynb"

In [ ]:
class DroneDetection(Dataset):
    def __init__(self, primaryDatasetPath):
        self.primaryDatasetPath = primaryDatasetPath

        self.entireDroneImageDataset, self.anchorBoxesDroneDataset, self.droneImagesClasses = self.get_data()

    def droneDatasetLength(self):
        return self.entireDroneImageDataset.size(dim=0)

    def droneDatasetRetrieveItem(self, droneDatasetindex):
        return self.entireDroneImageDataset[droneDatasetindex], self.anchorBoxesDroneDataset[droneDatasetindex], self.droneImagesClasses[droneDatasetindex]

    def droneDatasetRetrieveData(self):
        entireDroneImageDataset = []
        droneDatasetRetrieveIndices = []

        boxesDroneDataset, droneImagesClasses, droneDatasetImagePaths = droneDatasetJoinAnnotations(self.droneDetectionAnnotationPath, self.droneDetectionImageDirectory, self.droneDetectionImageSize)

        for m, droneDatasetImagePath in enumerate(droneDatasetImagePaths):
            #the object detection dataset skips past the drone's image classes and path if the path specified is not valid
            if (not droneDatasetImagePath) or (not os.path.exists(droneDatasetImagePath)):
                continue
            # the image is read and resized using the io library and the imread/resize methods.
            droneDatasetReadingImage = io.imread(droneDatasetImagePath)
            droneDatasetReadingImage = resize(droneDatasetReadingImage, self.droneDetectionImageSize)

            #The image is subsequently converted to torch tensor and reshaped so that channels come first

            droneDatasetImageTensor = torch.from_numpy(droneDatasetReadingImage).permute(2, 0, 1)

            #the specific drone class names are encoded as integers
            droneClasses = droneImagesClasses[m]
            droneDatasetIndices = torch.Tensor([self.droneDetectionIndexValue[droneDatasetName] for droneDatasetName in droneClasses])

            entireDroneImageDataset.append(droneDatasetImageTensor)
            droneDatasetRetrieveIndices.append(droneDatasetIndices)

            # the bounding boxes and classes are padded so that they are of equal size

        droneDatasetRetrievePaddedBoundingBoxes = droneDatasetPaddingValue(boxesDroneDataset, batch_first=True, padding_value=-1)
        droneDatasetPaddedClasses = droneDatasetPaddingValue(droneDatasetRetrieveIndices, batch_first=True, padding_value=-1)

            #all images across the entire dataset are stacked using the torch module
        droneDatasetStackedImage = torch.stack(entireDroneImageDataset, dim=0)

            #the stacked image for the drone dataset is returned and the retrieved padded bounding boxes and padded classes are sent to
            #the console for output to user as well
        return droneDatasetStackedImage.to(dtype=torch.float32), droneDatasetRetrievePaddedBoundingBoxes, droneDatasetPaddedClasses

In [ ]:
#The drone dataset model is fed into resnet50 a Fast-RCNN model using torchvision
droneDatasetModel = torchvision.models.resnet50(pretrained = True)


droneDatasetRequiredLayers = list(droneDatasetModel.children())[:8]
droneDatasetBackbone = nn.Sequential(*droneDatasetRequiredLayers)

#All the parameters for droneDataset are unfrozen

for droneDatasetParameter in droneDatasetBackbone.named_parameters():
  droneDatasetParameter[1].requires_grad = True

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#Drone dataset anchor is generated using the function below with the output size passed as parameter to it
def droneDetectionGeneratingAnchorCenter(droneDetectionOutputSize):
  #The output height and output width for the drone dataset are equivalent to the drone detection output size
  droneDatasetOutputHeight, droneDatasetOutputWeight = droneDetectionOutputSize

#The drone detection X Anchor Points and the drone detection Y anchor points are assigned to the Drone Dataset Output Width and Height,
#with 0.5 added for bias
  droneDetectionXAnchorPoints = torch.arange(0, droneDatasetOutputWeight) + 0.5
  droneDetectionYAnchorPoints = torch.arange(0, droneDatasetOutputHeight) + 0.5
#The X and Y anchor points are returned as output for the drone Detection anchor bounding box protocol.
  return droneDetectionXAnchorPoints, droneDetectionYAnchorPoints

In [ ]:
#The anchor box for the drone detection is generated with this function below with the X and Y anchor points as
#coordinates and the scale, ratio, and output size as additional parameters for this function
def droneDetectionGeneratingAnchorBox(droneDetectionXAnchorPoints, droneDetectionYAnchorPoints, droneDetectionAnchorScale, droneDetectionAnchorRatio, droneDetectionOutputSize):
  #the number of drone detection anchor bozes is equal to the length of the drone detection anchor scale multiplied
  #by the length of the drone detection anchor ratio

    droneDetectionNumberAnchorBoxes = len(droneDetectionAnchorScale) * len(droneDetectionAnchorRatio)
    droneDetectionAnchorBase = torch.zeros(1, droneDetectionXAnchorPoints.size(dim=0) \
                              , droneDetectionYAnchorPoints.size(dim=0), droneDetectionNumberAnchorBoxes, 4)

    for rowIndexXAnchorPoint, columnIndexXAnchorPoint in enumerate(droneDetectionXAnchorPoints):
        for rowIndexYAnchorPoint, columnIndexYAnchorPoint in enumerate(droneDetectionYAnchorPoints):
            droneDetectionAnchorBoxValue = torch.zeros((droneDetectionNumberAnchorBoxes, 4))
            q = 0
            for w, droneDetectionScale in enumerate(droneDetectionAnchorScale):
                for e, droneDetectionRatio in enumerate(droneDetectionAnchorRatio):
                    p = droneDetectionScale * droneDetectionRatio
                    d = droneDetectionScale

                    droneDetectionMinimumXValue = columnIndexXAnchorPoint - p / 2
                    droneDetectionMinimumYValue = columnIndexYAnchorPoint - d / 2
                    droneDetectionMaximumXValue = columnIndexXAnchorPoint + p / 2
                    droneDetectionMaximumYValue = columnIndexYAnchorPoint + d / 2

                    droneDetectionAnchorBoxValue[c, :] = torch.Tensor([droneDetectionMinimumXValue, droneDetectionMinimumYValue, droneDetectionMaximumXValue, droneDetectionMaximumYValue])
                    c += 1

            droneDetectionAnchorBase[:, rowIndexXAnchorPoint, rowIndexYAnchorPoint, :] = ops.clip_boxes_to_image(droneDetectionAnchorBoxValue, size=droneDetectionOutputSize)

    return droneDetectionAnchorBase

In [ ]:
#The drone dataset IOU matrix is retrieved with its batch size, total anchor boxes, and retrieved padded bounding boxes
def droneDatasetRetrieveIOUMatrix(droneDatasetBatchSize, droneDatasetEntireAnchorBoxes, droneDatasetRetrievePaddedBoundingBoxes):
#Flattening the entire anchor boxes and reshaping it with the batch size of the drone dataset will yield the flattened anchor boxes
#for the drone dataset
    droneDatasetFlattenAnchorBoxes = droneDatasetEntireAnchorBoxes.reshape(droneDatasetBatchSize, -1, 4)
#The complete anchor boxes will be the flattened anchor boxes for the drone
    droneDatasetTotalAnchorBoxes = droneDatasetFlattenAnchorBoxes.size(dim=1)
#The IOU matrix for the drone dataset will be invoked using the torch library with the batch size,
#total anchor boxes, and the retrieved padded bounding boxes
    droneDatasetIOUMatrix = torch.zeros((droneDatasetBatchSize, droneDatasetTotalAnchorBoxes, droneDatasetRetrievePaddedBoundingBoxes.size(dim=1)))
#Looping throuh the batch size for the drone dataset will create a bounding box value from the retrieve bounding box function
#and a flattened anchor boxes for w iterations and a IOU matrix with the anchor boxes and drone dataset
    for w in range(droneDatasetBatchSize):
        droneDatasetBoundingBoxValue = droneDatasetRetrievePaddedBoundingBoxes[w]
        droneDatasetAnchorBoxes = droneDatasetFlattenAnchorBoxes[w]
        droneDatasetIOUMatrix[w, :] = droneDatasetOPS.droneDatasetIOUBoxes(droneDatasetAnchorBoxes, droneDatasetBoundingBoxValue)

    return droneDatasetIOUMatrix

In [ ]:
def droneDatasetProjectBoundingBoxes(droneDatasetBoundingBoxes, droneDatasetWidthFactor, droneDatasetHeightFactor, droneDatasetMode='a2p'):
    #The two modes specified here are activation mode to pixel image and pixel mode to activation image
    assert droneDatasetMode in ['a2p', 'p2a']

#The drone dataset batch size is equal to its bounding boxes with a size of its dataset's collection
#of images
    droneDatasetBatchSize = droneDatasetBoundingBoxes.size(dim=0)
#The projected bounding boxes will be equal to the bounding boxes of the current drone dataset reshaped
    droneDatasetProjectedBoundingBoxes = droneDatasetBoundingBoxes.clone().reshape(droneDatasetBatchSize, -1, 4)
    droneDatasetInvalidBoundingBox = (droneDatasetProjectedBoundingBoxes == -1)
#If the drone dataset is of activation to pixel mode, multiply the bounding box indices by the width
#or height ratio
    if droneDatasetMode == 'a2p':
        droneDatasetProjectedBoundingBoxes[:, :, [0, 2]] *= droneDatasetWidthFactor
        droneDatasetProjectedBoundingBoxes[:, :, [1, 3]] *= droneDatasetHeightFactor
#If the drone dataset is of pixel to activation mode, divide the bounding box indices by the
#width or height ratio
    else:
        droneDatasetProjectedBoundingBoxes[:, :, [0, 2]] /= droneDatasetWidthFactor
        droneDatasetProjectedBoundingBoxes[:, :, [1, 3]] /= droneDatasetHeightFactor

    droneDatasetProjectedBoundingBoxes.masked_fill_(droneDatasetInvalidBoundingBox, -1)
    droneDatasetProjectedBoundingBoxes.resize_as_(droneDatasetBoundingBoxes)

    return droneDatasetProjectedBoundingBoxes

In [ ]:
#Calculate the drone dataset offset value by taking in the positive anchor coordinates and the
#bounding box mapping

def droneDatasetCalculateOffset(droneDatasetPositiveAnchorCoordinates, droneDatasetBoundingBoxMapping):
  #The drone dataset positive anchor coordinates is equal the converted coordinates
  #with the input and output format
    droneDatasetPositiveAnchorCoordinates = ops.box_convert(droneDatasetPositiveAnchorCoordinates, droneDatasetInputFormat='ewrt', droneDatasetOutputFormat='dfgsaa')
    droneDatasetBoundingBoxMapping = ops.box_convert(droneDatasetBoundingBoxMapping, droneDatasetInputFormat='ewrt', droneDatasetOutputFormat='dfgsaa')

    droneDatasetBoundingBoxXCoordinate, droneDatasetBoundingBoxYCoordinate, droneDatasetBoundingBoxWidth, droneDatasetBoundingBoxHeight = droneDatasetBoundingBoxMapping[:, 0], droneDatasetBoundingBoxMapping[:, 1], droneDatasetBoundingBoxMapping[:, 2], droneDatasetBoundingBoxMapping[:, 3]
    droneDatasetAnchorXCoordinate, droneDatasetAnchorYCoordinate, droneDatasetAnchorWidth, droneDatasetAnchorHeight = droneDatasetPositiveAnchorCoordinates[:, 0], droneDatasetPositiveAnchorCoordinates[:, 1], droneDatasetPositiveAnchorCoordinates[:, 2], droneDatasetPositiveAnchorCoordinates[:, 3]

    droneDatasetTorchXCoordinate_ = (droneDatasetBoundingBoxXCoordinate - droneDatasetAnchorXCoordinate)/droneDatasetAnchorWidth
    droneDatasetTorchYCoordinate_ = (droneDatasetBoundingBoxYCoordinate - droneDatasetAnchorYCoordinate)/droneDatasetAnchorHeight
    droneDatasetTorchWidth_ = torch.log(droneDatasetBoundingBoxWidth / droneDatasetAnchorWidth)
    droneDatasetTorchHeight = torch.log(droneDatasetBoundingBoxHeight / droneDatasetAnchorHeight)

    return torch.stack([droneDatasetTorchXCoordinate_, droneDatasetTorchYCoordinate_, droneDatasetTorchWidth_, droneDatasetTorchHeight], dim=-1)

In [ ]:
#This function retrieves the required anchors for the drone dataset

def droneDatasetRequiredAnchors(droneDatasetEntireAnchorBoxes, droneDatasetObtainBoundingBoxes, droneDatasetClasses, droneDatasetPositiveIOUThreshold=0.6, droneDatasetNegativeIOUThreshold=0.3):

#For the drone dataset, retrieve the size and shape parameter values with P and Y as the variables
    P, droneDetectionWidthAnchorMap, droneDetectionHeightAnchorMap, O, _ = droneDatasetEntireAnchorBoxes.shape
    Y= droneDatasetObtainBoundingBoxes.shape[1]
#The total number of anchor boxes in a single image is obtained
    droneDatasetTotalAnchorBoxes = O * droneDetectionWidthAnchorMap * droneDetectionHeightAnchorMap
#The IOU Matrix is obtained which has the IOU of every anchor box, in comparison
#with the collection of ground truth bounding boxes in an image
    droneDatasetIOUMatrix = droneDatasetRetrieveIOUMatrix(P, droneDatasetEntireAnchorBoxes, droneDatasetObtainBoundingBoxes)

    droneDatasetMaximumValuePerBoundingBox, _ = droneDatasetIOUMatrix.max(dim=1, keepdim=True)


    droneDatasetPositiveAnchorMask = torch.logical_and(droneDatasetIOUMatrix == droneDatasetMaximumValuePerBoundingBox, droneDatasetMaximumValuePerBoundingBox > 0)
    droneDatasetPositiveAnchorMask = torch.logical_or(droneDatasetPositiveAnchorMask, droneDatasetIOUMatrix > droneDatasetPositiveIOUThreshold)

    droneDatasetPositiveAnchorIndexSeparation = torch.where(droneDatasetPositiveAnchorMask)[0] # get separate indices in the batch

    droneDatasetPositiveAnchorMask = droneDatasetPositiveAnchorMask.flatten(start_dim=0, end_dim=1)
    droneDatasetPositiveAnchorIndex = torch.where(droneDatasetPositiveAnchorMask)[0]



    droneDatasetMaximumIOUPerAnchor, droneDatasetMaximumIOUPerAnchorIndex = droneDatasetIOUMatrix.max(dim=-1)
    droneDatasetMaximumIOUPerAnchor = droneDatasetMaximumIOUPerAnchor.flatten(start_dim=0, end_dim=1)


    droneDatasetConfidenceScores = droneDatasetMaximumIOUPerAnchor[droneDatasetPositiveAnchorIndex]

    droneDatasetBoundingClassesExpanded = droneDatasetClasses.view(P, 1, Y).expand(P, droneDatasetTotalAnchorBoxes, Y)
    droneDatasetBoundingClasses = torch.gather(droneDatasetBoundingClassesExpanded, -1, droneDatasetMaximumIOUPerAnchorIndex.unsqueeze(-1)).squeeze(-1)
    droneDatasetBoundingClasses = droneDatasetBoundingClasses.flatten(start_dim=0, end_dim=1)
    droneDatasetBoundingClassesPositive = droneDatasetBoundingClasses[droneDatasetPositiveAnchorIndex]


    droneDatasetBoundingBoxesExpanded = droneDatasetObtainBoundingBoxes.view(P, 1, Y, 4).expand(P, droneDatasetTotalAnchorBoxes, Y, 4)

    droneDatasetBoundingBoxes = torch.gather(droneDatasetBoundingClassesExpanded, -2, droneDatasetMaximumIOUPerAnchorIndex.reshape(P, droneDatasetTotalAnchorBoxes, 1, 1).repeat(1, 1, 1, 4))

    droneDatasetBoundingBoxes = droneDatasetBoundingBoxes.flatten(start_dim=0, end_dim=2)
    droneDatasetBoundingBoxesPositive = droneDatasetBoundingBoxes[droneDatasetPositiveAnchorIndex]

    droneDatasetAnchorBoxesFlattened = droneDatasetEntireAnchorBoxes.flatten(start_dim=0, end_dim=-2) # flatten all the anchor boxes
    droneDatasetPositiveAnchorCoordinates = droneDatasetAnchorBoxesFlattened[droneDatasetPositiveAnchorIndex]

    droneDatasetOffsetValue = droneDatasetCalculateOffset(droneDatasetPositiveAnchorCoordinates, droneDatasetBoundingBoxesPositive)


    droneDatasetNegativeAnchorMask = (droneDatasetMaximumIOUPerAnchor < droneDatasetNegativeIOUThreshold)
    droneDatasetNegativeAnchorIndex = torch.where(droneDatasetNegativeAnchorMask)[0]

    droneDatasetNegativeAnchorIndex = droneDatasetNegativeAnchorIndex[torch.randint(0, droneDatasetNegativeAnchorIndex.shape[0], (droneDatasetPositiveAnchorIndex.shape[0],))]
    droneDatasetNegativeAnchorCoordinates = droneDatasetAnchorBoxesFlattened[droneDatasetNegativeAnchorIndex]

    return droneDatasetPositiveAnchorIndex, droneDatasetNegativeAnchorIndex, droneDatasetConfidenceScores, droneDatasetOffsetValue, droneDatasetBoundingClassesPositive, \
         droneDatasetPositiveAnchorCoordinates, droneDatasetNegativeAnchorCoordinates, droneDatasetPositiveAnchorIndexSeparation

In [ ]:
class droneDatasetProposalModule(nn.Module):
    def __init__(self, droneDatasetInputFeatures, droneDatasetHiddenDimensions=512, droneDatasetNumberAnchors=9, droneDatasetProbabilityDropout=0.3):
        super().__init__()
        self.droneDatasetNumberAnchors = droneDatasetNumberAnchors
        self.droneDatasetConvolutionalLayer = nn.Conv2d(droneDatasetInputFeatures, droneDatasetHiddenDimensions, droneDatasetKernelSize=3, droneDatasetPadding=1)
        self.droneDatasetDropoutValue = nn.Dropout(droneDatasetProbabilityDropout)
        self.droneDatasetConfidenceHead = nn.Conv2d(droneDatasetHiddenDimensions, droneDatasetNumberAnchors, droneDatasetKernelSize=1)
        self.droneDatasetRegularHead = nn.Conv2d(droneDatasetHiddenDimensions, droneDatasetNumberAnchors * 4, droneDatasetKernelSize=1)

    def forward(self, feature_map, droneDatasetPositiveAnchorIndex=None, droneDatasetNegativeAnchorIndex=None, droneDatasetPositiveAnchorCoordinates=None):

        if droneDatasetPositiveAnchorIndex is None or droneDatasetNegativeAnchorIndex is None or droneDatasetPositiveAnchorCoordinates is None:
            droneDatasetOption = 'eval'
        else:
            droneDatasetOption = 'train'

        droneDatasetOutput = self.droneDatasetConvolutionalLayer(feature_map)
        droneDatasetOutput = F.relu(self.droneDatasetDropoutValue(droneDatasetOutput))

        droneDatasetRegularOffsetPrediction = self.droneDatasetRegularHead(droneDatasetOutput)
        droneDatasetConfidenceScoresPrediction = self.droneDatasetConfidenceHead(droneDatasetOutput)
#retrieving the confidence scores provided the drone dataset option is equivalent to train
        if droneDatasetOption == 'train':

            droneDatasetConfidenceScorePositive = droneDatasetConfidenceScoresPrediction.flatten()[droneDatasetPositiveAnchorIndex]
            droneDatasetConfidenceScoreNegative = droneDatasetConfidenceScoresPrediction.flatten()[droneDatasetNegativeAnchorIndex]
#the offset values for positive anchors is obtained
            droneDatasetOffsetPosition = droneDatasetRegularOffsetPrediction.contiguous().view(-1, 4)[droneDatasetPositiveAnchorIndex]
#generated proposals utilizing the offset values in previous step is obtained
            droneDatasetProposal = droneDatasetProposalModule(droneDatasetPositiveAnchorCoordinates, droneDatasetOffsetPosition)

            return droneDatasetConfidenceScorePositive, droneDatasetConfidenceScoreNegative, droneDatasetOffsetPosition, droneDatasetProposal
#if the drone dataset option is evaluation, return the drone dataset confidence scores prediction
#and regular offset prediction
        elif droneDatasetOption == 'eval':
            return droneDatasetConfidenceScoresPrediction, droneDatasetRegularOffsetPrediction

In [ ]:
def droneDatasetProposalCreation(droneDatasetAnchorValue, droneDatasetOffsetValue):

    droneDatasetAnchors = ops.box_convert(droneDatasetAnchors, droneDatasetInputFormat='ewrt', droneDatasetOutputFormat='cxcywh')

    droneDatasetProposalValue = torch.zeros_like(droneDatasetAnchors)
    droneDatasetProposalValue[:,0] = droneDatasetAnchors[:,0] + droneDatasetOffsetValue[:,0]*droneDatasetAnchors[:,2]
    droneDatasetProposalValue[:,1] = droneDatasetAnchors[:,1] + droneDatasetOffsetValue[:,1]*droneDatasetAnchors[:,3]
    droneDatasetProposalValue[:,2] = droneDatasetAnchors[:,2] * torch.exp(droneDatasetOffsetValue[:,2])
    droneDatasetProposalValue[:,3] = droneDatasetAnchors[:,3] * torch.exp(droneDatasetOffsetValue[:,3])

    droneDatasetProposalValue = ops.box_convert(droneDatasetProposalValue, droneDatasetInputFormat='ewrt', droneDatasetOutputFormat='cxcywh')

    return droneDatasetProposalValue

In [ ]:
class TwoStageDetector(nn.Module):
    def __init__(self, droneDatasetImageSize, droneDatasetOutputSize, droneDatasetOutputChannels, droneDatasetNumberClasses, droneDatasetROISize):
        super().__init__()
        self.regionProposalNetwork = droneDatasetRegionProposalNetwork(droneDatasetImageSize, droneDatasetOutputSize, droneDatasetOutputChannels)
        self.droneDatasetClassifier = ClassificationModule(droneDatasetOutputChannels, droneDatasetNumberClasses, droneDatasetROISize)

    def droneDatasetForward(self, droneDatasetImages, droneDatasetBoundingBoxes, droneDatasetClasses):
        droneDatasetRPNCumulativeLoss, droneDatasetFeatureMap, droneDatasetProposalValue, \
        droneDatasetPositiveAnchorIndexSeparation, droneDatasetClassPositive = self.droneDatasetRPNValue(droneDatasetImageValue, droneDatasetBoundingBox, droneDatasetClasses)

        droneDatasetPositiveProposalList = []
        droneDatasetBatchSize = droneDatasetImages.size(dim=0)
        for droneDatasetIndex in range(droneDatasetBatchSize):
            droneDatasetProposalIndices = torch.where(droneDatasetPositiveAnchorIndexSeparation == droneDetectionIndex)[0]
            droneDatasetProposalSeparation = droneDatasetProposalCreation[droneDatasetProposalIndex].detach().clone()
            droneDatasetPositiveProposalList.append(droneDatasetProposalSeparation)

        droneDatasetConfidenceLoss = self.droneDatasetClassifier(droneDatasetFeatureMap, droneDatasetPositiveProposalList, droneDatasetClassPositive)
        droneDatasetTotalLoss = droneDatasetConfidenceLoss + droneDatasetRPNLoss

        return droneDatasetTotalLoss

    def droneDatasetInferenceValue(self, droneDatasetImages, droneDatasetConfidenceValue=0.5, droneDatasetThreshold=0.7):
        droneDetectionBatchSize = droneDatasetImages.size(dim=0)
        droneDetectionFinalProposal, droneDetectionConfidenceScoreFinal, droneDetectionFeatureMap = self.droneDatasetRegionProposalNetwork.droneDatasetInferenceValue(droneDetectionImages, droneDatasetConfidenceValue, droneDatasetThreshold)
        droneDetectionConfidenceScore = self.classifier(droneDetectionFeatureMap, droneDetectionFinalProposal)

        droneDetectionConfidenceProbability = F.softmax(droneDetectionConfidenceScore, dim=-1)
        droneDetectionTotalClasses = torch.argmax(droneDetectionConfidenceProbability, dim=-1)

        droneDetectionFinalClasses = []
        w = 0
        for u in range(droneDetectionBatchSize):
            droneDetectionNumberProposals = len(droneDetectionFinalProposal[u]) # get the number of proposals for each image
            droneDetectionFinalClasses.append(droneDetectionTotalClasses[w: w+droneDetectionNumberProposals])
            w += droneDetectionNumberProposals

        return droneDetectionFinalProposal, droneDetectionConfidenceScore, droneDetectionFinalClasses

In [ ]:
def droneDetectionTrainingFastRCNN(droneDatasetModelValue, droneDatasetLearningRate, droneDatasetTrainingDataLoader, droneDatasetNumberEpochs):

  droneDatasetOptimizer = torch.optim.Adam(droneDatasetModelValue.parameters(), droneDatasetLR = droneDatasetLearningRate)

  droneDatasetModelValue.train()
  droneDatasetLossListCollection = []

  for v in droneDatasetLossFunction(range(droneDatasetNumberEpochs)):
    droneDatasetTotalLoss = 0
    for droneDatasetImageBatch, droneDatasetBoundingBoxesBatch, droneDatasetClassesBatch in droneDatasetTrainingDataLoader:

      droneDatasetLossValue = droneDatasetModelValue(droneDatasetImageBatch, droneDatasetBoundingBoxesBatch, droneDatasetClassesBatch)

      droneDatasetOptimizer.zero_grad()
      droneDatasetLossValue.backward()
      droneDatasetOptimizer.step()

      droneDatasetTotalLoss += droneDatasetLossValue.item()

    droneDatasetLossListCollection.append(droneDatasetTotalLoss)

  return droneDatasetLossListCollection

In [ ]:

import cv2

def droneDatasetFrameCapture(droneFirstVideoPath):

    droneFirstVideoPath = "/content/drive/MyDrive/DroneDetection/Drone Tracking 1.mp4"

    droneVideoObject = cv2.VideoCapture(droneFirstVideoPath)

    droneVideoCount = 0

    droneVideoSuccess = 1

    while droneVideoSuccess:

        droneVideoSuccess, droneVideoImageValue = droneVideoObject.read()

        #cv2.imwrite("frame%d.mp4" % droneVideoCount, droneFirstVideoPath)

        droneVideoCount += 1

if __name__ == '__main__':

    droneDatasetFrameCapture("C:\\Users\\sheel\\OneDrive\\Desktop\\Drone Tracking 1.mp4")

In [ ]:
import cv2

def droneDatasetFrameCapture(droneFirstVideoPath):

    droneSecondVideoPath = "/content/drive/MyDrive/DroneDetection/Drone Tracking 2.mp4"

    droneVideoObject = cv2.VideoCapture(droneSecondVideoPath)

    droneVideoCount = 0

    droneVideoSuccess = 1

    while droneVideoSuccess:

        droneVideoSuccess, droneVideoImageValue = droneVideoObject.read()

        #cv2.imwrite("frame%d.mp4" % droneVideoCount, droneSecondVideoPath)


        droneVideoCount += 1

if __name__ == '__main__':

    droneDatasetFrameCapture("C:\\Users\\sheel\\OneDrive\\Desktop\\Drone Tracking 2.mp4")